In [12]:
import pandas as pd
df = pd.read_csv("IE KPMG_Demand Forecast dataset.csv")
df

,year-week,ID_material,Units
0,2019-11,120010970,540.0
1,2019-12,120010970,750.0
2,2019-13,120010970,960.0
3,2019-14,120010970,720.0
4,2019-15,120010970,1620.0
...,...,...,...
2852,2022-35,120015996,948.0
2853,2022-36,120015996,600.0
2854,2022-37,120015996,1452.0
2855,2022-38,120015996,1920.0


In [13]:
df[df["ID_material"]==120015996].head(60)

,year-week,ID_material,Units
2617,2018-09,120015996,482.000
2618,2018-10,120015996,536.000
2619,2018-11,120015996,0.000
2620,2018-12,120015996,0.000
2621,2018-13,120015996,32.000
2622,2018-14,120015996,296.000
2623,2018-15,120015996,408.000
2624,2018-16,120015996,656.000
2625,2018-17,120015996,2524.000
2626,2018-18,120015996,244.000


In [14]:
# Filter the DataFrame to show only decimal values in the 'Units' column
decimal_values = df[df['Units'] % 1 != 0]

# Display the decimal values
print(decimal_values)

     year-week  ID_material     Units
2450   2019-30    120014488  1822.500
2668   2019-08    120015996     2.865
2743   2020-31    120015996  1358.500
2745   2020-33    120015996  1952.500


In [15]:
df.loc[2450, 'Units'] = 1822
df.loc[2668, 'Units'] = 2865
df.loc[2743, 'Units'] = 1358
df.loc[2745, 'Units'] = 1952

In [16]:
df['year-month-week'] = pd.to_datetime(df['year-week'] + '-1', format='%Y-%W-%w')
df['year-month'] = df['year-month-week'].dt.strftime('%Y-%m')
df = df.drop(columns="year-week")
df["year-month"] = pd.to_datetime(df['year-month'])

In [18]:
df["year-month"] = pd.to_datetime(df['year-month'])

In [19]:
df['MonthName'] = df['year-month'].dt.strftime('%B')
df['year-month'] = df['year-month'].astype(str)
df["Year"] = df["year-month"].str[:4]

season_mapping = {
    'January': 'Winter',
    'February': 'Winter',
    'March': 'Spring',
    'April': 'Spring',
    'May': 'Spring',
    'June': 'Summer',
    'July': 'Summer',
    'August': 'Summer',
    'September': 'Autumn',
    'October': 'Autumn',
    'November': 'Autumn',
    'December': 'Winter'
    }

df['Season'] = df['MonthName'].map(season_mapping)

In [20]:
import holidays

df['year-month-week'] = pd.to_datetime(df['year-month-week'])

# Create a list of US holidays
es_holidays = holidays.ES(years=df['year-month-week'].dt.year)

# Create a function that takes a week's start date and returns the number of holidays within that week
def count_holidays(start_date):
    week_dates = pd.date_range(start=start_date, periods=7)
    return sum(1 for date in week_dates if date in es_holidays)

# Apply the function to the "year-month-week" column to create the new column "Number_Of_Holidays_In_That_Week"
df['Number_Of_Holidays_In_That_Week_SPAIN'] = df['year-month-week'].apply(count_holidays)

df = df.drop(columns="year-month-week")
df = df.drop(columns="year-month")

df = df.groupby(['ID_material', 'Year', 'Season', 'MonthName']).agg({
    'Units': 'sum',
    'Number_Of_Holidays_In_That_Week_SPAIN': 'sum'
}).reset_index()

In [21]:
df

,ID_material,Year,Season,MonthName,Units,Number_Of_Holidays_In_That_Week_SPAIN
0,120004096,2017,Autumn,November,420.0,0
1,120004096,2017,Autumn,October,540.0,2
2,120004096,2017,Autumn,September,480.0,0
3,120004096,2017,Spring,April,360.0,1
4,120004096,2017,Spring,March,540.0,0
...,...,...,...,...,...,...
656,120015996,2022,Summer,August,4216.0,1
657,120015996,2022,Summer,July,3158.0,0
658,120015996,2022,Summer,June,5344.0,0
659,120015996,2022,Winter,February,3840.0,0


In [22]:
month_to_number = {'January': '01', 'February': '02', 'March': '03', 'April': '04', 'May': '05', 'June': '06',
                   'July': '07', 'August': '08', 'September': '09', 'October': '10', 'November': '11', 'December': '12'}
df['MonthNumber'] = df['MonthName'].map(month_to_number)
df['Year-Month'] = df['Year'].astype(str) + '-' + df['MonthNumber']

In [23]:
econ = pd.read_csv("GDPSPAIN.csv",sep=";",header=None)


In [24]:
econ = econ.rename(columns={1: 'GDP_Past_Quarter_Change'})
econ = econ.rename(columns={0: 'Year-Month'})


In [25]:
paro = pd.read_csv("DATAPAROESPANA.csv",sep=";")

In [26]:
paro

,Year-Month,LastKnownTerm
0,2017-01,4255.0
1,2017-02,4255.0
2,2017-03,4255.0
3,2017-04,4255.0
4,2017-05,4255.0
...,...,...
64,2022-05,3174.7
65,2022-06,3174.7
66,2022-07,2919.4
67,2022-08,2919.4


In [27]:
econ

,Year-Month,GDP_Past_Quarter_Change
0,2017-01,0.8
1,2017-02,0.8
2,2017-03,0.8
3,2017-04,0.8
4,2017-05,0.8
...,...,...
64,2022-05,-0.4
65,2022-06,-0.4
66,2022-07,2.5
67,2022-08,2.5


In [28]:
paro

,Year-Month,LastKnownTerm
0,2017-01,4255.0
1,2017-02,4255.0
2,2017-03,4255.0
3,2017-04,4255.0
4,2017-05,4255.0
...,...,...
64,2022-05,3174.7
65,2022-06,3174.7
66,2022-07,2919.4
67,2022-08,2919.4


In [29]:
df = df.merge(econ[['Year-Month', 'GDP_Past_Quarter_Change']], on='Year-Month', how='left')
df = df.merge(paro[['Year-Month', 'LastKnownTerm']], on='Year-Month', how='left')

In [30]:
df['Date'] = pd.to_datetime(df['Year'].astype(str) + ' ' + df['MonthName'], format='%Y %B')
df.sort_values('Date', inplace=True)
df.drop('Date', axis=1, inplace=True)
df = df.drop(columns="Year-Month")
df['GDP_Past_Quarter_Change'].fillna(0, inplace=True)


In [31]:
df['Months_Since_Material_Release'] = df.groupby('ID_material').cumcount()


In [32]:
df["Absolute_Previous GDP Quarter Change"] = df["GDP_Past_Quarter_Change"].abs()

In [33]:
# Drop the 'MonthNumber' column
df = df.drop('MonthNumber', axis=1)

# Rename the 'LastKnownTerm' column
df = df.rename(columns={'LastKnownTerm': 'PreviousUnemploymentQuarterAmmount'})
df = df.rename(columns={'Number_Of_Holidays_In_That_Week_SPAIN': 'Number_Of_Holidays_In_That_Month_SPAIN'})



In [34]:
df

,ID_material,Year,Season,MonthName,Units,Number_Of_Holidays_In_That_Month_SPAIN,GDP_Past_Quarter_Change,PreviousUnemploymentQuarterAmmount,Months_Since_Material_Release,Absolute_Previous GDP Quarter Change
11,120004096,2017,Winter,January,600.0,0,0.8,4255.0,0,0.8
10,120004096,2017,Winter,February,300.0,0,0.8,4255.0,1,0.8
4,120004096,2017,Spring,March,540.0,0,0.8,4255.0,2,0.8
3,120004096,2017,Spring,April,360.0,1,0.8,4255.0,3,0.8
5,120004096,2017,Spring,May,480.0,1,0.8,4255.0,4,0.8
...,...,...,...,...,...,...,...,...,...,...
511,120014486,2022,Autumn,September,7560.0,0,2.5,2919.4,59,2.5
60,120004096,2022,Autumn,September,600.0,0,2.5,2919.4,68,2.5
318,120010970,2022,Autumn,September,4290.0,0,2.5,2919.4,42,2.5
170,120009816,2022,Autumn,September,2880.0,0,2.5,2919.4,55,2.5


In [35]:
df.to_csv("monthly_data.csv",index=False)

In [36]:
df

,ID_material,Year,Season,MonthName,Units,Number_Of_Holidays_In_That_Month_SPAIN,GDP_Past_Quarter_Change,PreviousUnemploymentQuarterAmmount,Months_Since_Material_Release,Absolute_Previous GDP Quarter Change
11,120004096,2017,Winter,January,600.0,0,0.8,4255.0,0,0.8
10,120004096,2017,Winter,February,300.0,0,0.8,4255.0,1,0.8
4,120004096,2017,Spring,March,540.0,0,0.8,4255.0,2,0.8
3,120004096,2017,Spring,April,360.0,1,0.8,4255.0,3,0.8
5,120004096,2017,Spring,May,480.0,1,0.8,4255.0,4,0.8
...,...,...,...,...,...,...,...,...,...,...
511,120014486,2022,Autumn,September,7560.0,0,2.5,2919.4,59,2.5
60,120004096,2022,Autumn,September,600.0,0,2.5,2919.4,68,2.5
318,120010970,2022,Autumn,September,4290.0,0,2.5,2919.4,42,2.5
170,120009816,2022,Autumn,September,2880.0,0,2.5,2919.4,55,2.5
